# Demo notebook : SelfPlay with ONNX models

## Setup

In [175]:
MODE = "local"  # "colab" | "colab-dev" | "local"

In [176]:
if MODE == "colab":
    !pip install -q lczerolens
elif MODE == "colab-dev":
    !rm -r lczerolens
    !git clone https://github.com/Xmaster6y/lczerolens -b main
    !pip install -q ./lczerolens

## Download ONNX Models

In [177]:
!gdown 1yw2jzcdrGk3ao-nbVl42VUmCUF-6spfO -O maia1100.onnx
!gdown 1BFTmWwhILhAF4PJ2F3L15Fi5LPomCGFl -O maia1900.onnx

Downloading...
From: https://drive.google.com/uc?id=1yw2jzcdrGk3ao-nbVl42VUmCUF-6spfO
To: /mnt/d/Documents/0Polytech Sorbonne/MAIN/MAIN3/StageLip6/lczerolens/docs/source/notebooks/features/maia1100.onnx
100%|██████████████████████████████████████| 3.48M/3.48M [00:00<00:00, 35.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BFTmWwhILhAF4PJ2F3L15Fi5LPomCGFl
To: /mnt/d/Documents/0Polytech Sorbonne/MAIN/MAIN3/StageLip6/lczerolens/docs/source/notebooks/features/maia1900.onnx
100%|██████████████████████████████████████| 3.48M/3.48M [00:00<00:00, 37.2MB/s]


## Import Libraries

In [178]:
from lczerolens import LczeroBoard, LczeroModel
from lczerolens.sampling import SelfPlay, ModelSampler

## Load & Setup Model Samplers

In [179]:
maia_1100_model = LczeroModel.from_path("maia1100.onnx")
maia_1900_model = LczeroModel.from_path("maia1900.onnx")
white_sampler = ModelSampler(model=maia_1100_model, use_argmax=False)
black_sampler = ModelSampler(model=maia_1900_model, use_argmax=False)

## Initialize SelfPlay

In [180]:
self_play = SelfPlay(white=white_sampler, black=black_sampler)
logs = []


def report_fn(log, to_play):
    logs.append((log, to_play))

## Play Game

In [181]:
board = LczeroBoard()

game_moves, final_board = self_play.play(board=board, max_moves=100, report_fn=report_fn)

## Display Results

In [182]:
print("Game moves:", game_moves)
print("Game result:", final_board.result())

Game moves: [Move.from_uci('e2e4'), Move.from_uci('c7c5'), Move.from_uci('d1f3'), Move.from_uci('b8c6'), Move.from_uci('f1c4'), Move.from_uci('d7d6'), Move.from_uci('f3f7'), Move.from_uci('e8d7'), Move.from_uci('c4e6'), Move.from_uci('d7c7'), Move.from_uci('e6d5'), Move.from_uci('g8f6'), Move.from_uci('b1c3'), Move.from_uci('c6b4'), Move.from_uci('c3b5'), Move.from_uci('c7b8'), Move.from_uci('d5b3'), Move.from_uci('e7e5'), Move.from_uci('f7e6'), Move.from_uci('c8e6'), Move.from_uci('b3e6'), Move.from_uci('a7a6'), Move.from_uci('b5a3'), Move.from_uci('f6e4'), Move.from_uci('d2d3'), Move.from_uci('d8a5'), Move.from_uci('d3e4'), Move.from_uci('b4c2'), Move.from_uci('e1d1'), Move.from_uci('c2a3'), Move.from_uci('b2a3'), Move.from_uci('a5c3'), Move.from_uci('a1b1'), Move.from_uci('b8c7'), Move.from_uci('g1e2'), Move.from_uci('c3d3'), Move.from_uci('d1e1'), Move.from_uci('d3b1'), Move.from_uci('e2c3'), Move.from_uci('b1c1'), Move.from_uci('e1e2'), Move.from_uci('c1b2'), Move.from_uci('e2d3')